In [1]:
# Widgets
import bqviz as bqv
from ipywidgets import *
from bqwidgets import *
from bqplot import *
import bqplot
from bqplot.interacts import LassoSelector

# first step is to import the bql module
import bql
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import warnings
warnings.filterwarnings("ignore")

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

In [2]:
global global_mat
global iss_points

def bq_res_array_to_pd(res_array):
    a = res_array[0].df().reset_index()
    r_all = pd.DataFrame({'ID':a['ID']})
    for res in res_array:
        r_n = res.df().reset_index()
        r_all = r_all.merge(r_n, on = 'ID')
    return r_all

def download_names (tgt = None):
    global global_mat
    
    global_mat = pd.DataFrame()
    
    request = bql.Request(u.members(ac_index.value), {'Nombre': f.count(f.group(d.long_comp_name(), d.long_comp_name()))['value']})
    response = bq.execute(request)
    df_names = response[0].df().reset_index()
    ac_names.data = df_names['ID'].tolist()
    
    request = bql.Request(u.members(ac_index.value), {'Nombre': f.count(f.group(d.PAYMENT_RANK(), d.PAYMENT_RANK()))['value']})
    response = bq.execute(request)
    df_rank = response[0].df().reset_index()
    
    select_rank.options = df_rank['ID'].tolist()
    select_rank.value = df_rank['ID'].tolist()
    
def press_main_button (tgt = None):
    if ac_names.value != '':
        list_text.append(ac_names.value)               
        download_issuer()
        selector_list.options = list_text  
        plot_curves()

def download_issuer (tgt = None): 
    global global_mat
    
    issue_type = d.PAYMENT_RANK()

    cond_1 = f.in_(issue_type, list(select_rank.value))
    cond_2 = d.long_comp_name() == ac_names.value

    criteria_list = f.and_(cond_1,cond_2)
    filtered_univ = u.filter(u.members(ac_index.value), criteria_list)

    date_today = datetime.today()
    last_year = datetime(date_today.year - 1, 12, 31)
    last_month = datetime(date_today.year, date_today.month, 1) - relativedelta(days = 1)
    past_5d = date_today - relativedelta(days = 5)

    asw_today = d.spread(spread_type = 'ASW', fill = 'prev')
    asw_week = d.spread(spread_type = 'ASW', start = past_5d, fill = 'prev')
    asw_month = d.spread(spread_type = 'ASW', start = last_month, fill = 'prev')
    asw_year = d.spread(spread_type = 'ASW', start = last_year, fill = 'prev')

    matur = f.if_(f.or_(issue_type == 'Subordinated', f.or_(issue_type == 'Jr Subordinated', issue_type == 'Sr Subordinated')), d.NXT_CALL_DT(), d.maturity())

    request =  bql.Request(filtered_univ, {'Nombre Emisión': d.name(), 'Vencimiento': matur['value'],
                                           'ASW_TODAY': asw_today['value'], 'ASW_1WEEK': asw_week['value'], 'ASW_1MONTH': asw_month['value'], 'ASW_1YEAR': asw_year['value']})
    response = bq.execute(request)
    aux_df = bq_res_array_to_pd(response)
    aux_df['Nombre'] = ac_names.value
    aux_df['maturity_years'] = ((pd.to_datetime(aux_df['Vencimiento']) - date.today()) / 365) / timedelta(days = 1)
    aux_df = aux_df.sort_values('maturity_years')
    aux_df['Rank'] = list(range(len(aux_df)))  
    
    global_mat = global_mat.append(aux_df)    
    
def plot_curves (tgt = None):
    global global_mat
    global iss_points

    iss_curves = dict()
    iss_points = dict()
    issuers_list = list(selector_list.options)
    color_data = list(range(len(issuers_list)))

    for issuer_name, i_color in zip(issuers_list, color_data):

        print_df = global_mat.loc[global_mat['Nombre'].isin([issuer_name])]

        iss_curves[issuer_name] = bqplot.Lines(x = print_df['maturity_years'].tolist(), y = print_df['ASW_TODAY'].tolist(),
                                    scales={'x': sc_x, 'y': sc_y, 'color': sc_col}, display_legend=True, color = [i_color], marker = 'triangle-down', labels = [issuer_name])

        iss_points[issuer_name] = bqplot.Scatter(x = print_df['maturity_years'].tolist(), y = print_df['ASW_TODAY'].tolist(),
                                    scales={'x': sc_x, 'y': sc_y}, default_opacities = [0],
                                                names = print_df['Nombre Emisión'].tolist(), display_names = False)

    fig_line.marks = list(iss_curves.values()) + list(iss_points.values())
    lasso_sel.marks = list(iss_points.values())        

def plot_bars (tgt = None):
    global global_mat
    global iss_points
    
    df = pd.DataFrame()
    for i_issuer_name in list(selector_list.options):
        if len(iss_points[i_issuer_name].selected)>0:
            df = df.append(global_mat.loc[global_mat['Nombre'].isin([i_issuer_name]) & global_mat['Rank'].isin(iss_points[i_issuer_name].selected)][['Nombre Emisión', 'ASW_TODAY', 'ASW_1WEEK', 'ASW_1MONTH', 'ASW_1YEAR']], ignore_index=True)

    bar.x = []
    bar.y = []
    bar.labels = []
    bar.color = []    
    
    bar.x = df.loc[:, df.columns != 'Nombre Emisión'].columns
    bar.y = df.loc[:, df.columns != 'Nombre Emisión'].values
    bar.labels = df['Nombre Emisión'].tolist()
    bar.color = list(range(len(df)))
    
# Row1
ac_index = TickerAutoComplete(yellow_keys = ['Index'], max_results = 5, layout = Layout(width = '200px'))
button1 = Button(description='CARGAR',button_style='info')
button1.on_click(download_names)
row1 = HBox([ac_index, button1])

# Row 2
list_text = []
ac_names = AutoComplete(data = [], layout = Layout(width = '200px'))
button2 = Button(description='Añadir',button_style='warning')
button2.on_click(press_main_button)
selector_list = Select(options = list_text , layout=Layout(width='200px'))
selector_list.layout.height = '250px'

list_rank = []
select_rank = SelectMultiple(options = list_rank, value = list_rank, layout = Layout(height = '100px', width = '265px'))

row2 = HBox([ac_names, select_rank, button2])

sc_x = bqplot.LinearScale()
sc_y = bqplot.LinearScale()
sc_col = ColorScale(colors=['Red', 'Green'])

ax_x = bqplot.Axis(scale = sc_x, label = 'Vencimientos')
ax_y = bqplot.Axis(scale=sc_y, orientation = 'vertical', tick_format = '0.2f', label = 'ASW')

lasso_sel = LassoSelector()

fig_line = bqplot.Figure(marks = [], axes = [ax_x, ax_y], legend_location = 'bottom', interaction=lasso_sel)
fig_line.layout.width = '1500px'
fig_line.layout.height = '450px'

xs = bqplot.OrdinalScale()
ys = bqplot.LinearScale()
sc = bqplot.ColorScale()

bar = bqplot.Bars(x=[], y=[], scales={'x': xs, 'y': ys, 'color': sc}, type='grouped', color=[], padding=0.5, labels = [], display_legend=True)

xax = bqplot.Axis(scale=xs)
yax = bqplot.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label = 'ASW')

bar_plot = bqplot.Figure(marks=[bar], axes=[xax, yax])
bar_plot.layout.width = '1500px'
bar_plot.layout.height = '400px'

button3 = Button(description='VER ESTADÍSTICAS',button_style='success')
button3.layout.width = '1500px'
button3.on_click(plot_bars)

row3 = HBox([selector_list, VBox([fig_line, button3, bar_plot])])

VBox([row1, row2, row3])